#### Prevedere se il prezzo di un titolo salirà o scenderà

Build A Cryptocurrency Price Movement Classifier

https://www.youtube.com/watch?v=YeYmXNvFrpQ&ab_channel=ComputerScience

In [1]:
import numpy as np
from sklearn.model_selection import train_test_split
from datetime import date, datetime
#plt.style.use('fivethirtyeight')
import pandas_datareader.data as web
import pandas as pd

In [2]:
start_date = '2021-03-25'
end_date = '2021-07-02'
df = web.get_data_yahoo('BTC-USD', start_date, end_date) 

In [3]:
df.tail()

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2021-06-29,36542.109375,34252.484375,34475.558594,35867.777344,37901460044,35867.777344
2021-06-30,36074.757812,34086.152344,35908.386719,35040.835938,34059036099,35040.835938
2021-07-01,35035.984375,32883.781250,35035.984375,33572.117188,37838957079,33572.117188
2021-07-02,33939.589844,32770.679688,33549.601562,33897.046875,38728974942,33897.046875
2021-07-03,34909.261719,33402.695312,33854.421875,34668.546875,24383958643,34668.546875


Creiamo l'indicatore EMA (Exponential Moving Average)

In [5]:
def EMA(data, period=20, column = 'Close'):
    return data[column].ewm(span=period, adjust=False).mean()

Creiamo l'indicatore RSI (Relative Strength Index)

In [6]:
def RSI(data, period=14, column='Close'):
    delta = data[column].diff(1)
    delta = delta.dropna()
    up = delta.copy()
    down = delta.copy()
    up[up<0] = 0
    down[down>0] = 0
    data['up'] = up
    data['down'] = down
    AVG_Gain = EMA(data, period, column= 'up')
    AVG_Loss = abs(EMA(data, period, column= 'down'))
    RS = AVG_Gain / AVG_Loss
    RSI = 100.0 - (100.0 / (1.0 + RS))
    data['RSI' + str(period)] = RSI
    return data

In [7]:
RSI(df, 7)
RSI(df, 14)
RSI(df, 20)
df['EMA15'] = EMA(df,15)
df['EMA20'] = EMA(df,20)
df['EMA50'] = EMA(df,50)

In [8]:
df

,High,Low,Open,Close,Volume,Adj Close,up,down,RSI7,RSI14,RSI20,EMA15,EMA20,EMA50
Date,,,,,,,,,,,,,,
2021-03-25,53392.386719,50856.570312,52726.746094,51704.160156,67999812841,51704.160156,NaN,NaN,NaN,NaN,NaN,51704.160156,51704.160156,51704.160156
2021-03-26,55137.312500,51579.855469,51683.011719,55137.312500,56652197978,55137.312500,3433.152344,0.000000,100.000000,100.000000,100.000000,52133.304199,52031.127046,51838.793581
2021-03-27,56568.214844,54242.910156,55137.566406,55973.511719,47266542233,55973.511719,836.199219,0.000000,100.000000,100.000000,100.000000,52613.330139,52406.592253,52000.939391
2021-03-28,56610.312500,55071.113281,55974.941406,55950.746094,47686580918,55950.746094,0.000000,-22.765625,99.728156,99.886668,99.924836,53030.507133,52744.130714,52155.833771
2021-03-29,58342.097656,55139.339844,55947.898438,57750.199219,57625587027,57750.199219,1799.453125,0.000000,99.788694,99.897285,99.929468,53620.468644,53220.899143,52375.220652
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-06-29,36542.109375,34252.484375,34475.558594,35867.777344,37901460044,35867.777344,1433.441406,0.000000,68.280724,56.143919,52.132330,34831.635906,35291.533060,39103.892728
2021-06-30,36074.757812,34086.152344,35908.386719,35040.835938,34059036099,35040.835938,0.000000,-826.941406,56.681977,51.461639,49.136557,34857.785910,35267.657144,38944.557167
2021-07-01,35035.984375,32883.781250,35035.984375,33572.117188,37838957079,33572.117188,0.000000,-1468.718750,40.421641,43.950138,44.155556,34697.077320,35106.177148,38733.873246


Creiamo la colonna obiettivo per determinare se il prezzo di domani sarà più alto del prezzo di oggi. Se sì il valore è 1 altrimenti 0

In [9]:
df['Target'] = np.where(df['Close'].shift(-1)> df['Close'], 1, 0)

In [10]:
df.head()

,High,Low,Open,Close,Volume,Adj Close,up,down,RSI7,RSI14,RSI20,EMA15,EMA20,EMA50,Target
Date,,,,,,,,,,,,,,,
2021-03-25,53392.386719,50856.570312,52726.746094,51704.160156,67999812841,51704.160156,NaN,NaN,NaN,NaN,NaN,51704.160156,51704.160156,51704.160156,1
2021-03-26,55137.312500,51579.855469,51683.011719,55137.312500,56652197978,55137.312500,3433.152344,0.000000,100.000000,100.000000,100.000000,52133.304199,52031.127046,51838.793581,1
2021-03-27,56568.214844,54242.910156,55137.566406,55973.511719,47266542233,55973.511719,836.199219,0.000000,100.000000,100.000000,100.000000,52613.330139,52406.592253,52000.939391,0
2021-03-28,56610.312500,55071.113281,55974.941406,55950.746094,47686580918,55950.746094,0.000000,-22.765625,99.728156,99.886668,99.924836,53030.507133,52744.130714,52155.833771,1
2021-03-29,58342.097656,55139.339844,55947.898438,57750.199219,57625587027,57750.199219,1799.453125,0.000000,99.788694,99.897285,99.929468,53620.468644,53220.899143,52375.220652,1


Cancelliamo la prima riga di dati

In [11]:
df = df[1:]

In [12]:
df.head()

,High,Low,Open,Close,Volume,Adj Close,up,down,RSI7,RSI14,RSI20,EMA15,EMA20,EMA50,Target
Date,,,,,,,,,,,,,,,
2021-03-26,55137.312500,51579.855469,51683.011719,55137.312500,56652197978,55137.312500,3433.152344,0.000000,100.000000,100.000000,100.000000,52133.304199,52031.127046,51838.793581,1
2021-03-27,56568.214844,54242.910156,55137.566406,55973.511719,47266542233,55973.511719,836.199219,0.000000,100.000000,100.000000,100.000000,52613.330139,52406.592253,52000.939391,0
2021-03-28,56610.312500,55071.113281,55974.941406,55950.746094,47686580918,55950.746094,0.000000,-22.765625,99.728156,99.886668,99.924836,53030.507133,52744.130714,52155.833771,1
2021-03-29,58342.097656,55139.339844,55947.898438,57750.199219,57625587027,57750.199219,1799.453125,0.000000,99.788694,99.897285,99.929468,53620.468644,53220.899143,52375.220652,1
2021-03-30,59447.222656,57251.550781,57750.132812,58917.691406,54414116432,58917.691406,1167.492188,0.000000,99.822826,99.904016,99.932453,54282.621489,53763.450787,52631.788132,1


Facciamo la lista delle colonne da tenere

In [13]:
keep_columns = df.drop(['High', 'Low', 'Open', 'Volume','Adj Close',
                       'up', 'down','Close', 'Target'], axis = 1).columns
df[keep_columns].head()

,RSI7,RSI14,RSI20,EMA15,EMA20,EMA50
Date,,,,,,
2021-03-26,100.000000,100.000000,100.000000,52133.304199,52031.127046,51838.793581
2021-03-27,100.000000,100.000000,100.000000,52613.330139,52406.592253,52000.939391
2021-03-28,99.728156,99.886668,99.924836,53030.507133,52744.130714,52155.833771
2021-03-29,99.788694,99.897285,99.929468,53620.468644,53220.899143,52375.220652
2021-03-30,99.822826,99.904016,99.932453,54282.621489,53763.450787,52631.788132


In [14]:
X = df[keep_columns].values
Y = df['Target'].values

In [15]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)

Creiamo il modello - Classificatore "Random forest"

In [16]:
from sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier(n_estimators=1, criterion='entropy', random_state=1)
forest.fit(X_train, Y_train)

RandomForestClassifier(criterion='entropy', n_estimators=1, random_state=1)

Verifichiamo la bontà del modello sul data set di test

In [18]:
forest.score(X_test, Y_test)

0.3

In [19]:
forest_predictions = forest.predict(X_test)
forest_predictions

array([0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0])

In [20]:
Y_test

array([1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0])

In [21]:
1 - sum(abs(Y_test - forest_predictions))/len(Y_test)

0.30000000000000004